NLP from scratch. Classifying names with a character level RNN

A character level RNN reads words as a series of characters, outputting a prediction and 'hidden state at every step, feeding its previous hidden state into each next level step. Take the final prediction to be the output, which class the word belongs to.

PREPARE THE DATA

in 'data/names' there are 18 text files, with each file having one name per line

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob # used to find files recursively
import os

In [6]:
def findFiles(path): 
    return glob.glob(path)

print(findFiles('data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# turn a unicode string to plain ASCII
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
print(unicodeToAscii('Ślusàrski'))

# build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []


['data/names/Czech.txt', 'data/names/German.txt', 'data/names/Arabic.txt', 'data/names/Japanese.txt', 'data/names/Chinese.txt', 'data/names/Vietnamese.txt', 'data/names/Russian.txt', 'data/names/French.txt', 'data/names/Irish.txt', 'data/names/English.txt', 'data/names/Spanish.txt', 'data/names/Greek.txt', 'data/names/Italian.txt', 'data/names/Portuguese.txt', 'data/names/Scottish.txt', 'data/names/Dutch.txt', 'data/names/Korean.txt', 'data/names/Polish.txt']
Slusarski
